In [4]:
# Імпорт необхідних бібліотек
from surprise import Dataset
from surprise import SVD, SVDpp, NMF, KNNBasic, KNNWithMeans, KNNWithZScore, KNNBaseline, SlopeOne, CoClustering, BaselineOnly, NormalPredictor
from surprise.model_selection import cross_validate, train_test_split, GridSearchCV
from surprise.accuracy import rmse

# Завантаження датасету MovieLens (мінімальний набір)
data = Dataset.load_builtin('ml-100k')

# Розподіл даних на тренувальну та тестову вибірки
trainset, testset = train_test_split(data, test_size=0.2)

# Список алгоритмів для тестування
algorithms = {
    'SVD': SVD(),
    'SVD++': SVDpp(),
    'NMF': NMF(),
    'KNNBasic': KNNBasic(),
    'KNNWithMeans': KNNWithMeans(),
    'KNNWithZScore': KNNWithZScore(),
    'KNNBaseline': KNNBaseline(),
    'SlopeOne': SlopeOne(),
    'CoClustering': CoClustering(),
    'BaselineOnly': BaselineOnly(),
    'NormalPredictor': NormalPredictor()
}

# Проведення крос-валідації для кожної моделі та виведення результатів
results = {}
for algo_name, algo in algorithms.items():
    cv_results = cross_validate(algo, data, measures=['RMSE'], cv=5, verbose=False)
    avg_rmse = cv_results['test_rmse'].mean()
    results[algo_name] = avg_rmse
    print(f'Average RMSE for {algo_name}: {avg_rmse:.4f}')

# Використання GridSearchCV для підбору найкращих параметрів для SVD
param_grid = {
    'n_factors': [20, 50, 100],
    'n_epochs': [20, 50, 100],
    'lr_all': [0.002, 0.005],
    'reg_all': [0.02, 0.1]
}

grid_search = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=5)
grid_search.fit(data)

# Найкращі параметри та RMSE для SVD
print(f'Best SVD parameters: {grid_search.best_params["rmse"]}')
print(f'Best SVD RMSE: {grid_search.best_score["rmse"]:.4f}')

# Оцінка моделей на тестовій вибірці
best_model = grid_search.best_estimator['rmse']
best_model.fit(trainset)
predictions = best_model.test(testset)
test_rmse = rmse(predictions)

print(f'Test RMSE for the best SVD model: {test_rmse:.4f}')

# Додавання результатів SVD з GridSearch
results['SVD (GridSearch)'] = test_rmse

# Вибір оптимальної моделі на основі RMSE
optimal_model = min(results, key=results.get)
print(f'Optimal model: {optimal_model} with RMSE: {results[optimal_model]:.4f}')

Average RMSE for SVD: 0.9360
Average RMSE for SVD++: 0.9202
Average RMSE for NMF: 0.9642
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Average RMSE for KNNBasic: 0.9785
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Average RMSE for KNNWithMeans: 0.9499
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix.